In [1]:
import jupyter_black
from openai import OpenAI
import os
import json
from jinja2 import Template
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm
from dataclasses import dataclass
from IPython.display import Markdown


API_KEY = os.environ["DEEPSEEK_API_KEY"]
BASE_URL = "https://api.deepseek.com"
MODEL = "deepseek-chat"

jupyter_black.load()

In [2]:
def llm_call(prompt: str, with_json_output: bool = False) -> str | dict:
    client = OpenAI(api_key=API_KEY, base_url=BASE_URL)

    args = {
        "model": MODEL,
        "messages": [],
        "temperature": 0.0,
    }

    if with_json_output is True:
        json_prompt = """
        Output your response in JSON format with the keys specified in the prompt.
        Do not include any other text such as ```json or ```.
        The response should be directly parseable by json.loads.
        """.strip()
        args["messages"].append({"role": "system", "content": json_prompt})
        args["response_format"] = {"type": "json_object"}

    args["messages"].append({"role": "user", "content": prompt})
    response = client.chat.completions.create(**args)

    final_response = response.choices[0].message.content

    if with_json_output is True:
        return json.loads(final_response)

    return final_response

In [3]:
@dataclass
class Task:
    section_title: str
    rationale: str


@dataclass
class WorkerResult:
    section_title: str
    rationale: str
    result: str | None = None


class OrchestratorWorker:
    """Break down tasks and run them in parallel using worker LLMs."""

    def __init__(
        self,
        orchestrator_prompt: str,
        worker_prompt: str,
        synthesis_prompt: str,
        task: str,
    ):
        self.orchestrator_template = Template(orchestrator_prompt)
        self.worker_template = Template(worker_prompt)
        self.synthesis_template = Template(synthesis_prompt)
        self.task = task

    def run_single_worker(self, main_task: str, task: Task) -> WorkerResult:
        """Run a single worker and return the result."""

        worker_input = self.worker_template.render(
            original_task=main_task,
            section_title=task.section_title,
            rationale=task.rationale,
        )

        print(f"\n[WORKER INPUT] Section: {task.section_title}\n{worker_input}\n")
        worker_response = llm_call(worker_input, with_json_output=True)
        result = worker_response["response"]

        print(f"\n[WORKER OUTPUT] Section: {task.section_title}\n{result}\n")

        return WorkerResult(
            section_title=task.section_title,
            rationale=task.rationale,
            result=result,
        )

    def process(self, task: str) -> list[WorkerResult]:
        """Run orchestrator and execute all workers in parallel."""

        input_str = self.orchestrator_template.render(task=task)
        print(f"\n[ORCHESTRATOR INPUT]\n{input_str}\n")

        orchestrator_output = llm_call(input_str, with_json_output=True)

        print(f"[ORCHESTRATOR OUTPUT]\nAnalysis:\n{orchestrator_output['analysis']}")
        print(f"\nTasks:\n{orchestrator_output['tasks']}\n")

        worker_tasks = [Task(**t) for t in orchestrator_output["tasks"]]
        print(f"[INFO] Total tasks: {len(worker_tasks)}\n")

        with ThreadPoolExecutor() as executor:
            results = list(
                tqdm(
                    executor.map(
                        lambda t: self.run_single_worker(task, t), worker_tasks
                    ),
                    total=len(worker_tasks),
                    desc="Running workers",
                )
            )

        return results

    def synthesize(self, worker_results: list[WorkerResult]) -> str:
        """Generate final report by synthesizing all worker results."""

        synthesis_input = self.synthesis_template.render(
            original_task=self.task,
            worker_results=worker_results,
        )

        print(f"\n[SYNTHESIS INPUT]\n{synthesis_input}\n")

        response = llm_call(synthesis_input, with_json_output=True)
        return response["response"]

    def run(self) -> str:
        """Run the orchestrator and return the final report."""
        worker_results = self.process(self.task)
        synthesis_result = self.synthesize(worker_results)
        return synthesis_result

In [4]:
ORCHESTRATOR_PROMPT = """
You're tasked with creating a report on a specific topic.

Break this down into 3-5 distinct sections that could each be written with a different focus.

Report topic: {{ task }}

Return your response in this JSON format:

{
  "analysis": "Summarize your understanding of the reporting goal and explain why breaking it into multiple sections with different focuses would be valuable. Highlight how each section emphasizes a different priority (e.g., technical depth, accessibility, narrative).",
  "tasks": [
    {
      "section_title": "Technical Overview",
      "rationale": "Explain why this section is important and what it will cover.",
    },
    {
      "section_title": "Practical Implications",
      "rationale": "Explain why this section is important and what it will cover.",
    }
  ]
}
"""

WORKER_PROMPT = """
You're generating a section of a report.

Topic: {{ original_task }}
Section title: {{ section_title }}
Rationale: {{ rationale }}


Write the content for this section using **Markdown formatting** (bold, italics, lists, code blocks, etc.) where appropriate, but **do not include a header/title** — just the body content.

Return your response in this JSON format:

{
  "response": "Your report content here, using Markdown formatting but without any section headers."
}
"""


SYNTHESIS_PROMPT = """
You're synthesizing multiple report sections into one cohesive final report.

Original topic: {{ original_task }}

These are the completed sections (in order):

{% for result in worker_results %}
Section title: {{ result.section_title }}
---
{{ result.result }}
{% endfor %}

Your task is to merge the sections into a well-flowing report. Use **Markdown formatting** where helpful, but **do not repeat the section titles**.

Ensure:
- The tone and style feel consistent throughout.
- Transitions between sections are smooth.
- Any overlap is minimized.
- The final output reads like a single unified piece.

Return your response in this JSON format:

{
  "response": "Your final merged report here, in Markdown."
}
"""

In [5]:
task = "Make an in-depth report that explains why biking has not been adopted in the City of Lisbon, and what can be done to encourage it."
orchestrator = OrchestratorWorker(
    orchestrator_prompt=ORCHESTRATOR_PROMPT,
    worker_prompt=WORKER_PROMPT,
    synthesis_prompt=SYNTHESIS_PROMPT,
    task=task,
)
final_report = orchestrator.run()

print(final_report)


[ORCHESTRATOR INPUT]

You're tasked with creating a report on a specific topic.

Break this down into 3-5 distinct sections that could each be written with a different focus.

Report topic: Make an in-depth report that explains why biking has not been adopted in the City of Lisbon, and what can be done to encourage it.

Return your response in this JSON format:

{
  "analysis": "Summarize your understanding of the reporting goal and explain why breaking it into multiple sections with different focuses would be valuable. Highlight how each section emphasizes a different priority (e.g., technical depth, accessibility, narrative).",
  "tasks": [
    {
      "section_title": "Technical Overview",
      "rationale": "Explain why this section is important and what it will cover.",
    },
    {
      "section_title": "Practical Implications",
      "rationale": "Explain why this section is important and what it will cover.",
    }
  ]
}

[ORCHESTRATOR OUTPUT]
Analysis:
The reporting goal is 

Running workers:   0%|          | 0/5 [00:00<?, ?it/s]


[WORKER OUTPUT] Section: Cultural and Social Attitudes Towards Biking
In Lisbon, the cultural and social attitudes towards biking are mixed, with several factors contributing to its limited adoption. **Historical urban development** has prioritized cars and public transport, embedding a car-centric mindset among residents. This is compounded by the city's **hilly topography**, which many perceive as a natural barrier to biking, despite the potential for e-bikes to mitigate this issue.

*Key cultural perceptions include:*
- **Biking as a leisure activity**: Many Lisboetas view biking more as a recreational activity rather than a viable mode of daily transportation.
- **Safety concerns**: The lack of dedicated bike lanes and the dominance of cars make biking seem unsafe, especially for families and older adults.
- **Status symbol**: Cars are often seen as a status symbol, making biking less attractive to those who associate it with lower social status.

*To shift these attitudes, the fo

Running workers:  20%|██        | 1/5 [00:24<01:37, 24.26s/it]


[WORKER OUTPUT] Section: Community Engagement and Awareness Campaigns
To effectively promote biking in Lisbon, **community engagement and awareness campaigns** must be prioritized. These initiatives can transform public perception and encourage a shift towards cycling as a preferred mode of transportation. Below are key strategies:

- **Organize Community Bike Rides**: Regular events that invite residents to explore the city by bike, showcasing safe routes and the joy of cycling. *Example*: Monthly "Bike Lisbon" tours highlighting scenic and practical routes.

- **School and University Programs**: Partner with educational institutions to introduce biking initiatives, such as bike-to-school days or cycling safety workshops. **Goal**: Cultivate a biking culture from a young age.

- **Public Awareness Campaigns**: Utilize social media, local media, and public spaces to share the benefits of biking. Messages should focus on:
  - *Health advantages* (e.g., "Bike for a healthier Lisbon")
  

Running workers: 100%|██████████| 5/5 [00:24<00:00,  4.90s/it]


[WORKER OUTPUT] Section: Successful Case Studies and Potential Solutions
**Successful Case Studies**

- **Copenhagen, Denmark**: Often hailed as the biking capital of the world, Copenhagen has invested heavily in bike infrastructure, including dedicated bike lanes, traffic signals for cyclists, and bike bridges. *Key takeaway*: Prioritizing biking infrastructure as part of urban planning can significantly increase biking adoption.

- **Amsterdam, Netherlands**: Similar to Copenhagen, Amsterdam has a comprehensive network of bike paths, ample bike parking, and a culture that embraces biking. *Key takeaway*: Cultivating a biking culture alongside infrastructure development is crucial.

- **Bogotá, Colombia**: Through initiatives like Ciclovía, where streets are closed to cars every Sunday, Bogotá has encouraged biking among its citizens. *Key takeaway*: Temporary interventions can help shift public perception and encourage biking.

**Potential Solutions for Lisbon**

1. **Develop a Comp

The City of Lisbon faces significant challenges in adopting biking as a mainstream mode of transportation, primarily due to infrastructure limitations, cultural attitudes, and policy gaps. However, by addressing these issues through targeted strategies, Lisbon can foster a more bike-friendly environment.

**Current Infrastructure and Challenges**
Lisbon's urban landscape, characterized by steep hills and a lack of dedicated bike lanes, poses considerable barriers to biking. The absence of secure parking and poor integration with public transport further discourages potential cyclists. To overcome these obstacles, expanding the bike lane network, introducing e-bike sharing programs, and improving parking facilities are essential steps.

**Cultural and Social Attitudes Towards Biking**
Cultural perceptions in Lisbon often view biking as a leisure activity rather than a practical transportation option, compounded by safety concerns and the status associated with car ownership. Shifting th

In [7]:
Markdown(final_report)

The City of Lisbon faces significant challenges in adopting biking as a mainstream mode of transportation, primarily due to infrastructure limitations, cultural attitudes, and policy gaps. However, by addressing these issues through targeted strategies, Lisbon can foster a more bike-friendly environment.

**Current Infrastructure and Challenges**
Lisbon's urban landscape, characterized by steep hills and a lack of dedicated bike lanes, poses considerable barriers to biking. The absence of secure parking and poor integration with public transport further discourages potential cyclists. To overcome these obstacles, expanding the bike lane network, introducing e-bike sharing programs, and improving parking facilities are essential steps.

**Cultural and Social Attitudes Towards Biking**
Cultural perceptions in Lisbon often view biking as a leisure activity rather than a practical transportation option, compounded by safety concerns and the status associated with car ownership. Shifting these attitudes requires community engagement programs, infrastructure improvements, and incentives to highlight biking's benefits.

**Policy and Governance**
The lack of comprehensive biking policies and insufficient infrastructure have historically marginalized biking in Lisbon's transportation strategy. Developing a biking master plan, introducing incentives, and enhancing safety regulations are critical policy interventions to encourage biking adoption.

**Successful Case Studies and Potential Solutions**
Learning from cities like Copenhagen and Amsterdam, which have successfully integrated biking into their urban fabric, Lisbon can implement a comprehensive bike lane network, bike-sharing programs, and policies that favor biking. These measures, coupled with efforts to promote biking culture, can significantly increase biking adoption.

**Community Engagement and Awareness Campaigns**
Engaging the community through bike rides, educational programs, and public awareness campaigns can transform public perception. Incentive programs and platforms for citizen feedback will ensure that biking initiatives meet the community's needs, making biking an accessible and appealing option for all residents.

By addressing infrastructure challenges, shifting cultural attitudes, implementing supportive policies, learning from successful case studies, and engaging the community, Lisbon can overcome the barriers to biking adoption. This multifaceted approach will pave the way for a more sustainable and healthy urban mobility future in Lisbon.